In [1]:
from PIL import Image, ImageFont, ImageDraw
from datetime import datetime
import os
from glob import glob

from tqdm import tqdm
import os, os.path
try:
    from moviepy.editor import *
except:
    print('moviepy install required')
    !pip install moviepy
    from moviepy.editor import *

In [2]:
timelapse_params_json = {'photo_directory':r'D:\DCIM\100CANON',
                         'image_format_suffix':'.JPG',
                         'video_out_format_suffix':'.mp4',
                         'custom_output_directory':None,
                         'manual_time_correction':None,
                         'fps':15,
                         'video_width':1920,
                         'font_path':r'C:\Users\System-Pc\Desktop\arial.ttf',
                         'font_size':300,
                         'custom_event_name':None
                        }

In [6]:
def ensure_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)
    return
    
def get_date_taken(path,manual_time_correction=None):
    time_str = Image.open(path).getexif()[36867]
    if manual_time_correction is not None:
        ## Time correction here
        time_str = time_str
    return time_str

def markup_image(image_filename):
    image = Image.open(image_filename)  
    exif_timestr = image.getexif()[36867]
    exif_dt = datetime.strptime(exif_timestr, '%Y:%m:%d %H:%M:%S')

    if timelapse_params_json['manual_time_correction'] is not None:
        ## Time correction here
        exif_dt = exif_dt

    draw = ImageDraw.Draw(image)  
    # specified font size 
    font = ImageFont.truetype(timelapse_params_json['font_path'], timelapse_params_json['font_size'])  
    text = '{}'.format(exif_dt.strftime('%c'))
    # drawing text size 
    draw.text((5, 5), text, font = font, align ="right")  
    image.save(os.path.join(markup_photo_dir,image_filename))
    #image.show() 
    return

In [7]:
os.chdir(timelapse_params_json['photo_directory'])
photo_list = glob(r'*{}'.format(timelapse_params_json['image_format_suffix']))
if timelapse_params_json['custom_output_directory'] is None:
    dirName = datetime.strptime(get_date_taken(r'./{}'.format(photo_list[0])), '%Y:%m:%d %H:%M:%S').strftime('%d%b%y')
    output_directory = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),dirName)
    ensure_dir(output_directory)
else:
    output_directory = custom_output_directory
    ensure_dir(output_directory)

markup_photo_dir = os.path.join(output_directory,'marked_up_photos')
ensure_dir(markup_photo_dir)

#Create a standard filename
first_photo_dt = datetime.strptime(get_date_taken(r'./{}'.format(photo_list[0])), '%Y:%m:%d %H:%M:%S')
last_photo_dt = datetime.strptime(get_date_taken(r'./{}'.format(photo_list[-1])), '%Y:%m:%d %H:%M:%S')
startend_str = 'start{}_end{}'.format(first_photo_dt.strftime('%H%M_%d%b%y'),last_photo_dt.strftime('%H%M_%d%b%y'))
if timelapse_params_json['custom_event_name'] is None:
    output_video_name = '_'.join(['timelapse',startend_str])
else:
    output_video_name = '_'.join([timelapse_params_json['custom_event_name'],startend_str])

In [8]:
for filename in tqdm(photo_list):
    markup_image(filename)

100%|██████████| 1982/1982 [43:44<00:00,  1.32s/it] 


In [9]:
out_filename = os.path.join(output_directory,output_video_name+timelapse_params_json['video_out_format_suffix'])
file_names = [os.path.join(markup_photo_dir,name) for name in os.listdir(markup_photo_dir) if name[-4:] == timelapse_params_json['image_format_suffix']]

clip = ImageSequenceClip(file_names, fps = timelapse_params_json['fps'])
clip = clip.resize(width=timelapse_params_json['video_width'])
clip.write_videofile(out_filename, fps = timelapse_params_json['fps'])

Moviepy - Building video D:\DCIM\19Mar21\timelapse_start1156_19Mar21_end1728_19Mar21.mp4.
Moviepy - Writing video D:\DCIM\19Mar21\timelapse_start1156_19Mar21_end1728_19Mar21.mp4



Moviepy - Done !
Moviepy - video ready D:\DCIM\19Mar21\timelapse_start1156_19Mar21_end1728_19Mar21.mp4


In [149]:
clip = VideoFileClip("video.mp4")
fps= 1.0 # take one frame per second
nframes = clip.duration*fps # total number of frames used
total_image = sum(clip.iter_frames(fps,dtype=float,logger='bar'))
average_image = ImageClip(total_image/ nframes)
average_image.save_frame("average_test.png")

datetime.datetime(2021, 3, 9, 9, 10, 15, 636427)